In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from DDPG_Agent import DDPG_Agent
from QuadModel05 import QuadModel

settings = {
    'num_state': 12,
    'num_action':4,
    'DDPG':{
        'learning_rate_A':0.001,
        'learning_rate_C':0.005,
        'gamma':0.99,
        'batch_size':128,
        'replay_buffer_size':100000,
        'soft_update_rate':0.002,
        'num_fc1':64,
        'num_fc2':32,
        'fail':40,
    }
}

tf.reset_default_graph()

sess = tf.Session()
RL = DDPG_Agent(settings, sess)
    
saver = tf.train.Saver(max_to_keep=None)   # save all

def process_state(s):
    return np.reshape(s, [1, -1])

def generate_desired_vel():
    maxvel = 10
    co = 0.5
    velX = (2 * np.random.random() - 1) * maxvel * co
    velY = (2 * np.random.random() - 1) * maxvel * co
    velZ = (2 * np.random.random() - 1) * maxvel * co
    return np.array([velX, velY, velZ])

sigma = 0.1
rlog = []
traincount = 0

In [ ]:
for episode in range(10000):
    d_Vel = generate_desired_vel()
    quadmodel = QuadModel(d_Vel)
    s = quadmodel.observe()
    s = process_state(s)
    rsum = 0
    for step in range(500):
        action = sess.run(RL.A, feed_dict={RL.S:s})[0]
        # random exploration
        action = np.clip(np.random.normal(action, sigma), 0, 1)
        s1, r, d = quadmodel.step(action)
        s1 = process_state(s1)
        rsum += r
        RL.buffer_add(s, action, r, s1, d)
        if RL.buffer_ready:
            loss, rawQ = RL.train_it()
        if d: break
        s = s1
    # end of episode
    rlog.append(rsum)
    print 'episode:', episode, 'reward:', rsum